# Panda Example

In [4]:
import sys
sys.path.append('lib')

import scipy
import matplotlib.pyplot as plt
import numpy as np
import time, os
from IPython.display import clear_output

import pybullet as p
import pybullet_data

from pb_utils import set_q, vis_traj, create_primitives, get_joint_limits
from motion_planning import sampler, col_checker, interpolator, RRT

from functools import partial

DATA_PATH = '../../data'
robot_urdf = DATA_PATH + '/urdf/panda_arm.urdf'

%load_ext autoreload
%autoreload 2

np.set_printoptions(precision=4, suppress=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setup Pybullet

In [5]:
physics_client_id = p.connect(p.GUI)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)

In [6]:
p.resetSimulation()

### Setup Robot & environment

In [7]:
robot_id = p.loadURDF(fileName=robot_urdf)
dof = p.getNumJoints(robot_id)
pb_joint_indices = np.arange(7)
joint_limits = get_joint_limits(robot_id,pb_joint_indices)

plane_id = p.loadURDF('plane.urdf')
p.resetBasePositionAndOrientation(plane_id, (0,0,-.5), (0,0,0,1))

table_square_id = p.loadURDF('table_square/table_square.urdf')
p.resetBasePositionAndOrientation(table_square_id, (0.,0,-0.64), (0, 0, 0.7071068, 0.7071068))

table_id = p.loadURDF('table/table.urdf')
p.resetBasePositionAndOrientation(table_id, (.7,0,-0.5), (0, 0, 0.7071068, 0.7071068))

In [8]:
shelf_urdf = 'urdf/bookshelf_simple_collision.urdf'
shelf_id = p.loadURDF(fileName=shelf_urdf)
p.resetBasePositionAndOrientation(shelf_id, (-0.6,0.6,-0.5), (0, 0, 0, 1.))

#for visualizing the desired target
_,_,ball_id = create_primitives(radius=0.05)

In [13]:
set_q_std = partial(set_q,robot_id=robot_id, joint_indices=pb_joint_indices)

***

### Define the RRT

In [14]:
rob_simple_sampler = sampler(joint_limits)
rob_col_checker = col_checker(robot_id, pb_joint_indices, [ plane_id, shelf_id, table_id, table_square_id])
rob_interpolator = interpolator()

standard_rrt = RRT(7, rob_simple_sampler, rob_col_checker, rob_interpolator)

### Part 1: Planning from a random init config to a random goal config

#### Sample random init config

In [18]:
col_status = True
while col_status is True:
    q_init = rob_simple_sampler.sample().flatten()
    set_q_std(q_init)
    col_status = rob_col_checker.check_collision(q_init)

#### Sample random goal config

In [19]:
col_status = True
while col_status is True:
    q_goal = rob_simple_sampler.sample().flatten()
    set_q_std(q_goal)
    col_status = rob_col_checker.check_collision(q_goal)
    

In [20]:
path = standard_rrt.plan(np.array(q_init), np.array(q_goal))



Reached a goal state...
[array([-1.8742, -0.2899,  2.1983, -2.3785,  0.7594,  3.4354, -1.455 ]), array([-1.858 , -0.2643,  2.2573, -2.4137,  0.7104,  3.4693, -1.4395])]
[-1.858  -0.2643  2.2573 -2.4137  0.7104  3.4693 -1.4395]
Solution is found!
Planning...


In [21]:
vis_traj(traj, set_q_std, dt = 0.1)

### Part 2: Planning from an initial EE position to a goal EE position

#### Get init config using IK

In [28]:
col_status = True
while col_status is True:
    set_q_std(rob_simple_sampler.sample().flatten())
    q_init = np.array(p.calculateInverseKinematics(robot_id, 7,  (0., 0.5, 0.55)))
    set_q_std(q_init)
    col_status = rob_col_checker.check_collision(q_init)

#### Get goal config using IK

In [29]:
col_status = True
while col_status is True:
    set_q_std(rob_simple_sampler.sample().flatten())
    q_goal = np.array(p.calculateInverseKinematics(robot_id, 7,  (0.5, 0., 0.6)))
    set_q_std(q_goal)
    col_status = rob_col_checker.check_collision(q_goal)

#### Plan

In [30]:
traj = standard_rrt.plan(q_init, q_goal)


Trying to reach the goal state...
[array([-3.7585, -1.1814, -2.7459, -2.292 ,  2.0956,  1.3963, -2.4089]), array([-3.7926, -1.1699, -2.7483, -2.2976,  2.1776,  1.4122, -2.4409])]
[-3.7926 -1.1699 -2.7483 -2.2976  2.1776  1.4122 -2.4409]
Solution is found!
Planning...


In [31]:
vis_traj(traj, set_q_std)